

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/NER_TR.ipynb)

# **Detect entities in Turkish text**

## 1. Colab Setup, loading necassary libraries

In [1]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.1.2 spark-nlp

# Install Spark NLP Display lib
! pip install --upgrade -q spark-nlp-display

     |████████████████████████████████| 212.4 MB 69 kB/s 
     |████████████████████████████████| 145 kB 63.3 MB/s 
     |████████████████████████████████| 198 kB 35.8 MB/s 
     |████████████████████████████████| 95 kB 4.3 MB/s 
     |████████████████████████████████| 66 kB 6.3 MB/s 


In [2]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

## 2. Start the Spark session

In [3]:
spark = sparknlp.start()

In [4]:
spark.version

'3.1.2'

## 3. Select the DL model

In [5]:
# If you change the model, re-run all the cells below.
# Applicable embedding models: "glove_840B_300" and "bert_multi_cased"

# MODEL_NAME = 'turkish_ner_840B_300'
MODEL_NAME = 'turkish_ner_bert'

## 4. Some sample examples

In [6]:
# Enter examples to be transformed as strings in this list

text_list = ["""William Henry Gates III (28 Ekim 1955 doğumlu), Amerikalı bir iş adamı, \
yazılım geliştirici, yatırımcı ve hayırseverdir. En çok Microsoft şirketinin kurucu \
ortağı olarak bilinir. William Gates , Microsoft şirketindeki kariyeri boyunca başkan, \
icra kurulu başkanı, başkan ve yazılım mimarisi başkanı pozisyonlarında bulunmuş, \
aynı zamanda Mayıs 2014'e kadar en büyük bireysel hissedar olmuştur. O, 1970'lerin \
ve 1980'lerin mikrobilgisayar devriminin en tanınmış girişimcilerinden ve öncülerinden biridir. \
Seattle Washington'da doğup büyüyen William Gates, 1975'te New Mexico Albuquerque'de \
çocukluk arkadaşı Paul Allen ile Microsoft şirketini kurdu; dünyanın en büyük kişisel \
bilgisayar yazılım şirketi haline geldi. William Gates, Ocak 2000'de icra kurulu başkanı \
olarak istifa edene kadar şirketi başkan ve icra kurulu başkanı olarak yönetti ve \
daha sonra yazılım mimarisi başkanı oldu. 1990'ların sonlarında, William Gates rekabete \
aykırı olduğu düşünülen iş taktikleri nedeniyle eleştirilmişti. Bu görüş, çok sayıda \
mahkeme kararıyla onaylanmıştır. Haziran 2006'da William Gates, Microsoft şirketinde \
yarı zamanlı bir göreve ve 2000 yılında eşi Melinda Gates ile birlikte kurdukları özel \
hayır kurumu olan B&Melinda G. Vakfı'nda tam zamanlı çalışmaya geçeceğini duyurdu. \
Görevlerini kademeli olarak Ray Ozzie ve Craig Mundie'ye devretti. Şubat 2014'te \
Microsoft başkanlığından ayrıldı ve yeni atanan icra kurulu başkanı, Satya Nadella'yı \
desteklemek için teknoloji danışmanı olarak yeni bir göreve başladı.""",
    
"Mona Lisa, Leonardo tarafından yaratılan 16. yüzyıldan kalma bir yağlı boya tablodur. \
Tablo, Paris'teki Louvre Müzesi'nde sergileniyor.",
             
"""Facebook, 4 Şubat 2004 tarihinde TheFacebook adıyla başlatılan bir sosyal ağ hizmetidir. \
Mark Zuckerberg tarafından üniversite oda arkadaşları ve Harvard Üniversitesi öğrencileri \
Eduardo Saverin, Andrew McCollum, Dustin Moskovitz ve Chris Hughes ile birlikte kurulmuştur. \
Web sitesinin üyeliği başlangıçta kurucular tarafından Harvard öğrencileriyle sınırlıydı, ancak \
Boston bölgesindeki diğer kolejlere, Ivy-Ligine ve kademeli olarak ABD ve Kanada'daki çoğu üniversiteye genişletildi.""",
            
"""Geoffrey Everest Hinton, çoğu yapay sinir ağları üzerine yaptığı çalışmalarla tanınan \
İngiliz Kanadalı bir bilişsel psikolog ve bilgisayar bilimcisidir. 2013'ten beri zamanını \
Google ve Toronto Üniversitesi için ikiye ayırıyor. 2017 yılında, Toronto'da bulunan \
V. Enstitüsü'nün kurucu ortağı ve Bilimsel Konular Başdanışmanı oldu.""",
             
"""John Snow'a, Alaska'ya taşınmak istediğimi söylediğimde, orada bir Starbucks \
Cafe firması bulmakta zorlanacağım konusunda beni uyardı."""
            ]

## 5. Define Spark NLP pipeline

In [7]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

embeddings = None
public_ner = None

if MODEL_NAME == 'turkish_ner_840B_300' :
    embeddings = WordEmbeddingsModel.pretrained('glove_840B_300', "xx").\
                    setInputCols(["sentence", 'token']).\
                    setOutputCol("embeddings").\
                    setCaseSensitive(True)

    public_ner = NerDLModel.pretrained('turkish_ner_840B_300', 'tr') \
              .setInputCols(["sentence", "token", "embeddings"]) \
              .setOutputCol("ner")
    
elif MODEL_NAME == 'turkish_ner_bert' :
    embeddings = BertEmbeddings.pretrained('bert_multi_cased', 'xx') \
        .setInputCols(["sentence", "token"])\
        .setOutputCol("embeddings")

    public_ner = NerDLModel.pretrained('turkish_ner_bert', 'tr') \
              .setInputCols(["sentence", "token", "embeddings"]) \
              .setOutputCol("ner")

ner_converter = NerConverter() \
                .setInputCols(["sentence", "token", "ner"]) \
                  .setOutputCol("ner_chunk")

nlp_pipeline = Pipeline(stages=[ documentAssembler, sentenceDetector,
                                 tokenizer,
                                 embeddings,
                                 public_ner,
                                 ner_converter
                                 ])

bert_multi_cased download started this may take some time.
Approximate size to download 638.6 MB
[OK!]
turkish_ner_bert download started this may take some time.
Approximate size to download 15.5 MB
[OK!]


## 6. Run the pipeline

In [8]:
empty_df = spark.createDataFrame([['']]).toDF('text')
pipeline_model = nlp_pipeline.fit(empty_df)
df = spark.createDataFrame(pd.DataFrame({'text': text_list}))
result = pipeline_model.transform(df)

In [13]:
import pandas as pd


df = pd.read_csv('dev_wiki.csv', delimiter = "\t")
df = df.drop('Unnamed: 0', axis = 1)
df.rename(columns = {'sentence':'text', 'word_labels':'label'}, inplace = True)
print(df.columns.values)

['text' 'label']


In [14]:
df_spark = spark.createDataFrame(df)
result = pipeline_model.transform(df_spark)

In [15]:
result.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|               label|            document|            sentence|               token|          embeddings|                 ner|           ner_chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Corina Casanova ,...|I-PER I-PER O I-L...|[{document, 0, 48...|[{document, 0, 48...|[{token, 0, 5, Co...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 14, C...|
|Corina Casanova b...| I-PER I-PER O O O O|[{document, 0, 43...|[{document, 0, 43...|[{token, 0, 5, Co...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 0, 14, C...|
|İsviçre Dışişleri...|I-ORG I-ORG I-ORG...|[{document, 0, 72...|[{document, 0, 72...|[{token, 0, 6, İs...|[{word_embeddings...|[{named_entity, 0...|[{

In [16]:
result.select('label', 'ner.result').show(truncate=40)

+----------------------------------------+----------------------------------------+
|                                   label|                                  result|
+----------------------------------------+----------------------------------------+
|               I-PER I-PER O I-LOC O O O|       [B-PER, I-PER, O, B-LOC, O, O, O]|
|                     I-PER I-PER O O O O|              [B-PER, I-PER, O, O, O, O]|
|     I-ORG I-ORG I-ORG O I-ORG O O O O O|[B-ORG, I-ORG, I-ORG, O, B-LOC, O, O,...|
|                         I-LOC O O O O O|                  [B-LOC, O, O, O, O, O]|
|O I-PER O O O O O O O O O O I-PER O O...|[O, B-PER, I-PER, O, O, O, O, O, O, O...|
|O O O I-PER O O O O O O O O O O O O O...|[B-LOC, O, B-PER, I-PER, O, B-PER, O,...|
|                             O O O O O O|                      [O, O, O, O, O, O]|
|I-PER I-PER I-PER I-PER O O O O O O O...|[B-PER, I-PER, I-PER, I-PER, O, O, O,...|
|     O O O O O O I-LOC O O O O O O O O O|[O, O, O, O, B-ORG, O, O, O, B-ORG

In [17]:


import pandas as pd

df_result = result.select('label','ner.result').toPandas()

df_result

,label,result
0,I-PER I-PER O I-LOC O O O,"[B-PER, I-PER, O, B-LOC, O, O, O]"
1,I-PER I-PER O O O O,"[B-PER, I-PER, O, O, O, O]"
2,I-ORG I-ORG I-ORG O I-ORG O O O O O,"[B-ORG, I-ORG, I-ORG, O, B-LOC, O, O, O, O, O]"
3,I-LOC O O O O O,"[B-LOC, O, O, O, O, O]"
4,O I-PER O O O O O O O O O O I-PER O O O O O O,"[O, B-PER, I-PER, O, O, O, O, O, O, O, O, O, B..."
...,...,...
2995,O O O O O O O O O O O O O O O O O O O O O O O ...,"[B-PER, I-PER, O, O, O, O, O, O, O, O, O, B-PE..."
2996,I-PER O O O I-LOC O O O O O O,"[B-ORG, O, O, O, B-LOC, O, O, O, O, O, O]"
2997,O O O O O O O,"[O, O, O, O, O, O, O]"
2998,I-PER O O O O O O O I-PER O O O O O O O I-LOC O O,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [18]:
df_lbls = []
df_preds = []
for cnt, i in enumerate(df_result.label):
  a = i.strip().split(" ")
  if len(a) == len(df_result.iloc[cnt]["result"]):
    df_lbls.append(a)
    df_preds.append(df_result.iloc[cnt]["result"])


In [19]:
len(df_lbls), len(df_preds)

(2711, 2711)

In [25]:
y_p = [ item for sublist in df_preds for item in sublist ]
y_t = [ item for sublist in df_lbls for item in sublist ]

In [26]:
y_p

['B-PER',
 'I-PER',
 'O',
 'B-LOC',
 'O',
 'O',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'I-ORG',
 'I-ORG',
 'O',
 'B-LOC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'B-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-PER',
 'I-PER',
 'I-PER',
 'I-PER',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'O',
 'O',
 'O',
 'B-PER',
 'I-PER',
 'O',
 'O',
 'B-PER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'O',
 'O',
 'O',
 'B-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'I-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'I-ORG',
 'I-ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ORG',
 'I-ORG',
 'O',
 'O',
 'O',
 'O',
 'B-PER',
 'O',
 'O',
 'O',
 'B-ORG'

In [27]:
y_t_clean = []
y_p_clean = []


for cnt, i in enumerate(y_t):
  if i != "O":
    y_t_clean.append(i)
    y_p_clean.append(y_p[cnt])


In [28]:
from sklearn.metrics import classification_report
print(classification_report(y_t_clean, y_p_clean))

              precision    recall  f1-score   support

       B-LOC       0.00      0.00      0.00         0
       B-ORG       0.00      0.00      0.00         0
       B-PER       0.00      0.00      0.00         0
       I-LOC       0.92      0.09      0.17      1341
       I-ORG       0.70      0.37      0.48       797
       I-PER       0.95      0.32      0.48      1235
           O       0.00      0.00      0.00         0

    accuracy                           0.24      3373
   macro avg       0.37      0.11      0.16      3373
weighted avg       0.88      0.24      0.36      3373



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
test

precision    recall  f1-score   support

         LOC       0.98      0.78      0.87       578
           O       0.00      0.00      0.00         0
         ORG       0.95      0.85      0.90       657
         PER       0.98      0.92      0.95      1057

    accuracy                           0.87      2292
   macro avg       0.73      0.64      0.68      2292
weighted avg       0.97      0.87      0.92      2292


dev





precision    recall  f1-score   support

         LOC       0.96      0.86      0.90       538
           O       0.00      0.00      0.00         0
         ORG       0.95      0.83      0.89       792
         PER       0.99      0.90      0.94      1234

    accuracy                           0.87      2564
   macro avg       0.72      0.65      0.68      2564
weighted avg       0.97      0.87      0.92      2564


